In [2]:
!pip install music21

  Using cached more_itertools-10.8.0-py3-none-any.whl.metadata (39 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 13.4 MB/s  0:00:01m0:00:0100:01
Using cached more_itertools-10.8.0-py3-none-any.whl (69 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [music21]m3/4 [music21]


In [ ]:
import os
import json
import random
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
from collections import Counter

from music21 import converter, instrument, note, chord, stream

import tensorflow as tf
from tensorflow.keras import layers, regularizers, callbacks, Model

from sklearn.model_selection import train_test_split

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [2]:
@dataclass
class ModelConfig:
    name: str
    sequence_length: int = 50
    batch_size: int = 64
    embedding_dim: int = 64
    lstm_units: int = 256
    num_lstm_layers: int = 1
    dropout: float = 0.3
    recurrent_dropout: float = 0.0
    l1_reg: float = 1e-5
    learning_rate: float = 1e-3
    max_epochs: int = 50
    patience: int = 5
    min_note_freq: int = 3 

In [16]:
DATA_DIR = "data/midi"             
OUTPUT_DIR = "outputs/music_lstm" 

In [3]:
def load_midi_file_paths(data_dir: str, max_files: Optional[int] = None) -> List[str]:
    midi_files = []
    for root, _, files in os.walk(data_dir):
        for f in files:
            if f.lower().endswith((".mid", ".midi")):
                midi_files.append(os.path.join(root, f))
    midi_files.sort()
    if max_files is not None:
        midi_files = midi_files[:max_files]
    print(f"Found {len(midi_files)} MIDI files.")
    return midi_files

In [ ]:

def extract_notes_from_midi(file_paths: List[str]) -> List[str]:
    notes = []
    for i, fp in enumerate(file_paths):
        print(f"[{i+1}/{len(file_paths)}] Parsing {fp} ...")
        try:
            midi = converter.parse(fp)
        except Exception as e:
            print(f"  Skipping {fp} due to parse error: {e}")
            continue

        parts = instrument.partitionByInstrument(midi)
        if parts: 
            part_stream = None
            for p in parts.parts:
                if "Piano" in p.partName if p.partName else False:
                    part_stream = p
                    break
            if part_stream is None:
                part_stream = parts.parts[0]
        else:
            part_stream = midi.flat

        for element in part_stream.recurse():
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                pitches = ".".join(sorted(str(n) for n in element.pitches))
                notes.append(pitches)
    print(f"Extracted {len(notes)} note/chord events.")
    return notes

In [ ]:
def build_vocabulary(notes: List[str], min_freq: int = 3) -> Tuple[Dict[str, int], Dict[int, str]]:
    freq = Counter(notes)
    vocab_tokens = [tok for tok, c in freq.items() if c >= min_freq]
    vocab_tokens.sort()

    token_to_int = {"<PAD>": 0, "<UNK>": 1}
    idx = 2
    for tok in vocab_tokens:
        token_to_int[tok] = idx
        idx += 1

    int_to_token = {i: t for t, i in token_to_int.items()}

    print(f"Vocabulary size (including PAD & UNK): {len(token_to_int)}")
    return token_to_int, int_to_token

In [6]:
def encode_notes(notes: List[str], token_to_int: Dict[str, int]) -> np.ndarray:
    unk_id = token_to_int["<UNK>"]
    encoded = np.array([token_to_int.get(tok, unk_id) for tok in notes], dtype=np.int32)
    print(f"Encoded note sequence length: {len(encoded)}")
    return encoded

In [ ]:
def create_sequences(
    encoded_notes: np.ndarray,
    seq_length: int
) -> Tuple[np.ndarray, np.ndarray]:
    
    inputs = []
    targets = []

    for i in range(0, len(encoded_notes) - seq_length):
        inputs.append(encoded_notes[i : i + seq_length])
        targets.append(encoded_notes[i + seq_length])

    X = np.array(inputs, dtype=np.int32)
    y = np.array(targets, dtype=np.int32)

    print(f"Created {X.shape[0]} sequences of length {seq_length}.")
    return X, y

In [ ]:
def build_lstm_model(config: ModelConfig, vocab_size: int) -> Model:
    reg = regularizers.l1(config.l1_reg)

    inputs = layers.Input(shape=(config.sequence_length,), name="input_tokens")

    x = layers.Embedding(
        input_dim=vocab_size,
        output_dim=config.embedding_dim,
        mask_zero=True,
        name="embedding"
    )(inputs)

    if config.num_lstm_layers == 1:
        x = layers.LSTM(
            config.lstm_units,
            return_sequences=False,
            dropout=config.dropout,
            recurrent_dropout=config.recurrent_dropout,
            kernel_regularizer=reg,
            name="lstm_1"
        )(x)
    elif config.num_lstm_layers == 2:
        x = layers.LSTM(
            config.lstm_units,
            return_sequences=True,
            dropout=config.dropout,
            recurrent_dropout=config.recurrent_dropout,
            kernel_regularizer=reg,
            name="lstm_1"
        )(x)
        x = layers.LSTM(
            config.lstm_units // 2, 
            return_sequences=False,
            dropout=config.dropout,
            recurrent_dropout=config.recurrent_dropout,
            kernel_regularizer=reg,
            name="lstm_2"
        )(x)
    else:
        raise ValueError("num_lstm_layers must be 1 or 2 for this project.")

    x = layers.Dense(
        config.lstm_units // 2,
        activation="relu",
        kernel_regularizer=reg,
        name="dense_projection"
    )(x)

    outputs = layers.Dense(vocab_size, activation="softmax", name="output")(x)

    model = Model(inputs=inputs, outputs=outputs, name=f"lstm_music_{config.name}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"]
    )

    model.summary()
    return model

In [9]:
def make_tf_dataset(
    X: np.ndarray,
    y: np.ndarray,
    batch_size: int,
    shuffle: bool = True
) -> tf.data.Dataset:
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
def train_with_configs(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_val: np.ndarray,
    y_val: np.ndarray,
    vocab_size: int,
    configs: List[ModelConfig]
) -> Tuple[Model, ModelConfig, Dict[str, float]]:
    best_model = None
    best_config = None
    best_val_loss = float("inf")
    best_metrics = {}

    for cfg in configs:
        print("\n" + "=" * 80)
        print(f"Training config: {cfg.name}")
        print(cfg)
        print("=" * 80 + "\n")

        model = build_lstm_model(cfg, vocab_size)

        train_ds = make_tf_dataset(X_train, y_train, batch_size=cfg.batch_size, shuffle=True)
        val_ds = make_tf_dataset(X_val, y_val, batch_size=cfg.batch_size, shuffle=False)

        ckpt_path = os.path.join(OUTPUT_DIR, f"model_{cfg.name}.keras")

        cb_early = callbacks.EarlyStopping(
            monitor="val_loss",
            patience=cfg.patience,
            restore_best_weights=True
        )
        cb_ckpt = callbacks.ModelCheckpoint(
            filepath=ckpt_path,
            monitor="val_loss",
            save_best_only=True
        )
        cb_lr = callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=2, verbose=1
        )

        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=cfg.max_epochs,
            callbacks=[cb_early, cb_ckpt, cb_lr],
            verbose=2
        )

        val_loss, val_acc = model.evaluate(val_ds, verbose=0)
        print(f"[{cfg.name}] Final val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

        metrics = {
            "val_loss": float(val_loss),
            "val_acc": float(val_acc),
            "history": {
                "loss": [float(x) for x in history.history["loss"]],
                "val_loss": [float(x) for x in history.history["val_loss"]],
                "sparse_categorical_accuracy": [
                    float(x) for x in history.history["sparse_categorical_accuracy"]
                ],
                "val_sparse_categorical_accuracy": [
                    float(x) for x in history.history["val_sparse_categorical_accuracy"]
                ]
            }
        }

        metrics_path = os.path.join(OUTPUT_DIR, f"metrics_{cfg.name}.json")
        with open(metrics_path, "w") as f:
            json.dump(metrics, f, indent=2)
        print(f"Saved metrics to {metrics_path}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model
            best_config = cfg
            best_metrics = metrics

    print("\nBest config:", best_config)
    print(f"Best validation loss: {best_val_loss:.4f}")
    return best_model, best_config, best_metrics


In [ ]:
def sample_from_logits(logits: np.ndarray, temperature: float = 1.0) -> int:

    if temperature <= 0:
        return int(np.argmax(logits))

    logits = logits.astype(np.float64)
    logits = logits / temperature
    probs = np.exp(logits) / np.sum(np.exp(logits))
    return int(np.random.choice(len(probs), p=probs))

In [ ]:
def generate_continuation(
    model: Model,
    seed_sequence: List[int],
    int_to_token: Dict[int, str],
    token_to_int: Dict[str, int],
    num_generate: int = 200,
    temperature: float = 1.0,
    sequence_length: int = 50
) -> List[str]:
    model_input = list(seed_sequence)
    generated_tokens = []

    pad_id = token_to_int["<PAD>"]

    for i in range(num_generate):
        if len(model_input) < sequence_length:
            input_seq = [pad_id] * (sequence_length - len(model_input)) + model_input
        else:
            input_seq = model_input[-sequence_length:]

        input_arr = np.array([input_seq], dtype=np.int32)
        preds = model.predict(input_arr, verbose=0)[0]  # (vocab_size,)
        next_id = sample_from_logits(np.log(preds + 1e-9), temperature=temperature)

        generated_tokens.append(int_to_token.get(next_id, "<UNK>"))
        model_input.append(next_id)

    return generated_tokens

In [ ]:
def tokens_to_midi(
    tokens: List[str],
    output_path: str,
    quarter_length: float = 0.5
) -> None:

    out_stream = stream.Stream()
    offset = 0.0

    for tok in tokens:
        if tok in ("<PAD>", "<UNK>"):
            offset += quarter_length
            continue

        if "." in tok:
            pitches = tok.split(".")
            chord_notes = []
            for p in pitches:
                try:
                    n = note.Note(p)
                    n.storedInstrument = instrument.Piano()
                    chord_notes.append(n)
                except Exception:
                    pass
            if chord_notes:
                c = chord.Chord(chord_notes)
                c.offset = offset
                out_stream.append(c)
        else:
            try:
                n = note.Note(tok)
                n.offset = offset
                n.storedInstrument = instrument.Piano()
                out_stream.append(n)
            except Exception:
                pass

        offset += quarter_length

    out_stream.write("midi", fp=output_path)
    print(f"Saved generated MIDI to {output_path}")

In [ ]:
def main():

    midi_files = load_midi_file_paths(DATA_DIR, max_files=None) 
    notes = extract_notes_from_midi(midi_files)

    base_config = ModelConfig(name="base")
    token_to_int, int_to_token = build_vocabulary(notes, min_freq=base_config.min_note_freq)
    encoded = encode_notes(notes, token_to_int)

    X, y = create_sequences(encoded, seq_length=base_config.sequence_length)

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.1, random_state=SEED, shuffle=True
    )
    print(f"Train sequences: {X_train.shape[0]}, Val sequences: {X_val.shape[0]}")

    vocab_size = len(token_to_int)

    configs = [
        ModelConfig(
            name="small_lstm",
            sequence_length=50,
            batch_size=64,
            embedding_dim=48,
            lstm_units=128,
            num_lstm_layers=1,
            dropout=0.3,
            recurrent_dropout=0.1,
            l1_reg=1e-5,
            learning_rate=1e-3,
            max_epochs=40,
            patience=5,
            min_note_freq=base_config.min_note_freq
        ),
        ModelConfig(
            name="medium_lstm",
            sequence_length=50,
            batch_size=64,
            embedding_dim=64,
            lstm_units=256,
            num_lstm_layers=1,
            dropout=0.4,
            recurrent_dropout=0.1,
            l1_reg=1e-5,
            learning_rate=1e-3,
            max_epochs=50,
            patience=6,
            min_note_freq=base_config.min_note_freq
        ),
        ModelConfig(
            name="two_layer_lstm",
            sequence_length=50,
            batch_size=64,
            embedding_dim=64,
            lstm_units=256,
            num_lstm_layers=2,
            dropout=0.3,
            recurrent_dropout=0.1,
            l1_reg=1e-5,
            learning_rate=5e-4,
            max_epochs=60,
            patience=7,
            min_note_freq=base_config.min_note_freq
        ),
    ]

    best_model, best_config, best_metrics = train_with_configs(
        X_train, y_train, X_val, y_val, vocab_size, configs
    )

    best_model_path = os.path.join(OUTPUT_DIR, f"best_model_{best_config.name}.keras")
    best_model.save(best_model_path)
    print(f"Saved best model to {best_model_path}")

    mapping_path = os.path.join(OUTPUT_DIR, "vocab_mappings.json")
    with open(mapping_path, "w") as f:
        json.dump(
            {
                "token_to_int": token_to_int,
                "int_to_token": int_to_token,
                "best_config": asdict(best_config),
                "best_metrics": {k: best_metrics[k] for k in ("val_loss", "val_acc")}
            },
            f,
            indent=2
        )
    print(f"Saved vocabulary and config metadata to {mapping_path}")

    idx = np.random.randint(0, X_train.shape[0])
    seed_seq = X_train[idx].tolist()

    generated_tokens = generate_continuation(
        model=best_model,
        seed_sequence=seed_seq,
        int_to_token=int_to_token,
        token_to_int=token_to_int,
        num_generate=200,
        temperature=1.0,
        sequence_length=best_config.sequence_length
    )

    decoded_seed = [int_to_token.get(i, "<UNK>") for i in seed_seq]
    full_tokens = decoded_seed + generated_tokens

    midi_output_path = os.path.join(OUTPUT_DIR, f"demo_continuation_{best_config.name}.mid")
    tokens_to_midi(full_tokens, midi_output_path, quarter_length=0.5)

    print("Done. Use the saved metrics, activations, and MIDI to build your report.")

In [17]:
if __name__ == "__main__":
    main()

Found 295 MIDI files.
[1/295] Parsing data/midi/albeniz/alb_esp1.mid ...


/opt/anaconda3/envs/ds_environment/lib/python3.10/site-packages/music21/midi/translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, data=b'Copyright \xa9 2001 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(


[2/295] Parsing data/midi/albeniz/alb_esp2.mid ...
[3/295] Parsing data/midi/albeniz/alb_esp3.mid ...
[4/295] Parsing data/midi/albeniz/alb_esp4.mid ...
[5/295] Parsing data/midi/albeniz/alb_esp5.mid ...
[6/295] Parsing data/midi/albeniz/alb_esp6.mid ...
[7/295] Parsing data/midi/albeniz/alb_se1.mid ...
[8/295] Parsing data/midi/albeniz/alb_se2.mid ...
[9/295] Parsing data/midi/albeniz/alb_se3.mid ...
[10/295] Parsing data/midi/albeniz/alb_se4.mid ...
[11/295] Parsing data/midi/albeniz/alb_se5.mid ...
[12/295] Parsing data/midi/albeniz/alb_se6.mid ...
[13/295] Parsing data/midi/albeniz/alb_se7.mid ...
[14/295] Parsing data/midi/albeniz/alb_se8.mid ...
[15/295] Parsing data/midi/bach/bach_846.mid ...


/opt/anaconda3/envs/ds_environment/lib/python3.10/site-packages/music21/midi/translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=8, data=b'Copyright 2004 by Bernd Kr\xfcger.'>; getting generic Instrument
  warnings.warn(


[16/295] Parsing data/midi/bach/bach_847.mid ...


...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_tokens        │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 64)    │     10,560 │ input_tokens[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 50)        │          0 │ input_tokens[0][… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 50, 256)   │    328,704 │ embedding[0][0],  │
│                     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 128)       │    197,120 │ lstm_1[0][0],     │
│                     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_projection    │ (None, 128)       │     16,512 │ lstm_2[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 165)       │     21,285 │ dense_projection… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 574,181 (2.19 MB)

 Trainable params: 574,181 (2.19 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/60
39/39 - 289s - 7s/step - loss: 4.8895 - sparse_categorical_accuracy: 0.0687 - val_loss: 4.6855 - val_sparse_categorical_accuracy: 0.0473 - learning_rate: 5.0000e-04
Epoch 2/60
39/39 - 286s - 7s/step - loss: 4.5670 - sparse_categorical_accuracy: 0.0634 - val_loss: 4.6140 - val_sparse_categorical_accuracy: 0.0473 - learning_rate: 5.0000e-04
Epoch 3/60
39/39 - 285s - 7s/step - loss: 4.5294 - sparse_categorical_accuracy: 0.0590 - val_loss: 4.6172 - val_sparse_categorical_accuracy: 0.0473 - learning_rate: 5.0000e-04
Epoch 4/60
39/39 - 286s - 7s/step - loss: 4.5193 - sparse_categorical_accuracy: 0.0541 - val_loss: 4.6064 - val_sparse_categorical_accuracy: 0.0473 - learning_rate: 5.0000e-04
Epoch 5/60
39/39 - 286s - 7s/step - loss: 4.5104 - sparse_categorical_accuracy: 0.0578 - val_loss: 4.5875 - val_sparse_categorical_accuracy: 0.0473 - learning_rate: 5.0000e-04
Epoch 6/60
39/39 - 292s - 7s/step - loss: 4.5007 - sparse_categorical_accuracy: 0.0651 - val_loss: 4.5709 - val_sparse_c

In [10]:
import json
import pandas as pd

def load_metrics(path: str, name: str) -> dict:
    with open(path, "r") as f:
        d = json.load(f)

    history = d["history"]

    final_train_loss = history["loss"][-1]
    final_val_loss = history["val_loss"][-1]
    final_train_acc = history["sparse_categorical_accuracy"][-1]
    final_val_acc = history["val_sparse_categorical_accuracy"][-1]

    best_val_loss = min(history["val_loss"])
    best_epoch = history["val_loss"].index(best_val_loss) + 1
    val_acc_at_best = history["val_sparse_categorical_accuracy"][best_epoch - 1]

    return {
        "model": name,
        "final_train_loss": final_train_loss,
        "final_val_loss": final_val_loss,
        "final_train_acc": final_train_acc,
        "final_val_acc": final_val_acc,
        "best_val_loss": best_val_loss,
        "best_epoch": best_epoch,
        "val_acc_at_best_epoch": val_acc_at_best,
    }

def main():
    rows = []
    path = '/Users/tanmayswami/Downloads/Northwestern/MSDS 458/Final Project/outputs/music_lstm/'
    rows.append(load_metrics(f"{path}metrics_small_lstm.json", "small_lstm"))
    rows.append(load_metrics(f"{path}metrics_medium_lstm.json", "medium_lstm"))
    rows.append(load_metrics(f"{path}metrics_two_layer_lstm.json", "two_layer_lstm"))

    df = pd.DataFrame(rows)
    df = df.sort_values("best_val_loss").reset_index(drop=True)
    # print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

    return df

In [12]:
df = main()

In [14]:
df

,model,final_train_loss,final_val_loss,final_train_acc,final_val_acc,best_val_loss,best_epoch,val_acc_at_best_epoch
0,medium_lstm,0.368674,1.385466,0.933737,0.738182,1.376033,47,0.741818
1,small_lstm,1.010676,1.718880,0.717576,0.541818,1.718880,40,0.541818
2,two_layer_lstm,1.419490,1.992827,0.539394,0.410909,1.992827,60,0.410909


In [13]:
df.to_json()

'{"model":{"0":"medium_lstm","1":"small_lstm","2":"two_layer_lstm"},"final_train_loss":{"0":0.3686742783,"1":1.010676384,"2":1.4194897413},"final_val_loss":{"0":1.3854655027,"1":1.7188800573,"2":1.9928267002},"final_train_acc":{"0":0.9337373972,"1":0.7175757289,"2":0.5393939614},"final_val_acc":{"0":0.7381818295,"1":0.5418182015,"2":0.4109090865},"best_val_loss":{"0":1.3760325909,"1":1.7188800573,"2":1.9928267002},"best_epoch":{"0":47,"1":40,"2":60},"val_acc_at_best_epoch":{"0":0.7418181896,"1":0.5418182015,"2":0.4109090865}}'